In [65]:
_ = open(r'E:\Uni_Projects\3DMeshes\MedMeshCNN\checkpoints\final_40meshes_10train\meshes\Hannover_AM_left_initial_proc_0.obj')
test_mesh = _.readlines()
_.close()
print(test_mesh[:2])

['v 65.605723 -0.486512 20.889799\n', 'v 65.703002 -0.657188 21.357752\n']


In [77]:
vertices, faces, edges, edge_groups = [], [], [], []
for line in test_mesh:
    sp = line.split(' ')
    if sp[0].strip() == 'v':
        vertices.append([float(val) for val in sp[1:]])
    if sp[0].strip() == 'f':
        faces.append([int(val) for val in sp[1:]])        
    if sp[0].strip() == 'e':
        edges.append([int(val) for val in sp[1:3]])     
        edge_groups.append(int(sp[3]))

In [78]:
len(vertices), len(faces)

(9561, 19200)

In [79]:
len(edges), len(edge_groups)

(28800, 28800)

In [70]:
_ = open(r"E:\Uni_Projects\3DMeshes\Preprocessing3DMeshes\final_set_50meshes\edges\Hannover_AM_left_initial_proc.edges")
target_mesh = _.readlines()
_.close()
print(target_mesh[:2])

['0,1\n', '1,2\n']


In [71]:
target_edges = []
for line in target_mesh:
    sp = line.split(',')
    target_edges.append([int(val)+1 for val in sp])

In [72]:
min(target_edges), max(target_edges), len(target_edges)

([1, 2], [9560, 9561], 28800)

In [73]:
min(edges), max(edges), len(edges)

([1, 2], [9560, 9561], 28800)

In [37]:
_ = open(r"E:\Uni_Projects\3DMeshes\Preprocessing3DMeshes\final_set_50meshes\test\Hannover_AM_left_initial_proc.obj")
target_mesh_obj = _.readlines()
_.close()
print(target_mesh_obj[:2])

['####\n', '#\n']


In [38]:
target_vertices = []
for line in target_mesh_obj:
    sp = line.split(' ')
    if sp[0].strip() == 'v':
        target_vertices.append([float(val) for val in sp[1:]])

In [39]:
len(target_vertices)

9561

In [43]:
count = 0
for v, t_v in zip(vertices, target_vertices):
    if v != t_v:
        count += 1

In [44]:
count

0

In [76]:
_ = open(r"E:/Uni_Projects/3DMeshes/Preprocessing3DMeshes/final_set_50meshes/seg/Hannover_AM_left_initial_proc.eseg")
target_edge_groups = _.readlines()
_.close()
print(target_edge_groups[:2])

['0\n', '0\n']


In [80]:
target_edge_groups = [int(i) for i in target_edge_groups]    

In [86]:
tp, tn, fp, fn = 0, 0, 0, 0
for edg, tar in zip(edge_groups, target_edge_groups):
    if edg == 0 and tar == 0:
        tp += 1
    elif edg == 0 and tar == 1:
        fp += 1
    elif edg == 1 and tar == 0:
        fn += 1
    elif edg == 1 and tar == 1:
        tn += 1

In [84]:
count/len(edge_groups)

0.8734722222222222

In [87]:
tp / (tp+fp+fn)

0.8559911476446411

In [88]:
tp, tn, fp, fn = 0, 0, 0, 0
for edg, tar in zip(edge_groups, target_edge_groups):
    if edg == 1 and tar == 1:
        tp += 1
    elif edg == 1 and tar == 0:
        fp += 1
    elif edg == 0 and tar == 1:
        fn += 1
    elif edg == 0 and tar == 0:
        tn += 1

In [89]:
tp / (tp+fp+fn)

0.4896358543417367

In [90]:
def get_edge_groups(mesh):
    _ = open(r'%s'%mesh)
    test_mesh = _.readlines()
    _.close()

    # vertices, faces, edges, edge_groups = [], [], [], []
    edge_groups = []
    for line in test_mesh:
        sp = line.split(' ')     
        if sp[0].strip() == 'e':
            # edges.append([int(val) for val in sp[1:3]])     
            edge_groups.append(int(sp[3]))    
    return edge_groups

In [114]:
def get_target_edge_groups(mesh):
    _ = open(r'%s'%mesh)
    target_edge_groups = _.readlines()
    _.close()
    return [int(i) for i in target_edge_groups]    

In [119]:
def get_metrics(edge_groups, target_edge_groups):
    # IOU - Class 0
    tp, tn, fp, fn = 0, 0, 0, 0
    for edg, tar in zip(edge_groups, target_edge_groups):
        if edg == 0 and tar == 0:
            tp += 1
        elif edg == 0 and tar == 1:
            fp += 1
        elif edg == 1 and tar == 0:
            fn += 1
        elif edg == 1 and tar == 1:
            tn += 1
            
    iou_0 = tp / (tp+fp+fn)
    
    # IOU - Class 1
    tp, tn, fp, fn = 0, 0, 0, 0
    for edg, tar in zip(edge_groups, target_edge_groups):
        if edg == 1 and tar == 1:
            tp += 1
        elif edg == 1 and tar == 0:
            fp += 1
        elif edg == 0 and tar == 1:
            fn += 1
        elif edg == 0 and tar == 0:
            tn += 1
            
    iou_1 = tp / (tp+fp+fn)   
    
    # Accuracy
    acc = 0
    for edg, tar in zip(edge_groups, target_edge_groups):
        if edg == tar:
            acc += 1
            
    acc = acc/len(edge_groups)
    
    return iou_0, iou_1, acc

In [120]:
edge_groups = get_edge_groups(r'E:\Uni_Projects\3DMeshes\MedMeshCNN\checkpoints\final_40meshes_10train\meshes\Hannover_AM_left_initial_proc_0.obj')
target_edge_groups = get_target_edge_groups('E:/Uni_Projects/3DMeshes/Preprocessing3DMeshes/final_set_50meshes/seg/Hannover_AM_left_initial_proc.eseg')

In [121]:
get_metrics(edge_groups, target_edge_groups)

(0.8559911476446411, 0.4896358543417367, 0.8734722222222222)

In [126]:
test_meshes_text = \
"""
Hannover_AM_left_initial_proc.obj
Hannover_AZ_initial_proc.obj
Hannover_DK_initialSegmentation_proc.obj
Hannover_GDPT_left_initial_2_proc.obj
Hannover_MJ_links_initial_proc.obj
Hannover_SE_Segm1_initial_proc.obj
Hannover_WLAD_initial_proc.obj
HRA_20180325_2_initial_proc.obj
Kiel_BB_Pat20_seg_proc.obj
Kiel_BB_Pat2_initial.cut_proc.obj
"""

In [128]:
import re
test_meshes = [re.sub('\.obj', '', mesh.strip()) for mesh in test_meshes_text.split('\n') if mesh != '']
test_meshes

['Hannover_AM_left_initial_proc',
 'Hannover_AZ_initial_proc',
 'Hannover_DK_initialSegmentation_proc',
 'Hannover_GDPT_left_initial_2_proc',
 'Hannover_MJ_links_initial_proc',
 'Hannover_SE_Segm1_initial_proc',
 'Hannover_WLAD_initial_proc',
 'HRA_20180325_2_initial_proc',
 'Kiel_BB_Pat20_seg_proc',
 'Kiel_BB_Pat2_initial.cut_proc']

In [132]:
all_metrics = []
for mesh in test_meshes:
    chckpoint_mesh = r'E:\Uni_Projects\3DMeshes\MedMeshCNN\checkpoints\final_40meshes_10train\meshes\%s_0.obj'%mesh
    eseg_file = r'E:/Uni_Projects/3DMeshes/Preprocessing3DMeshes/final_set_50meshes/seg/%s.eseg'%mesh
    
    edge_groups = get_edge_groups(chckpoint_mesh)
    target_edge_groups = get_target_edge_groups(eseg_file)
    
    metrics = get_metrics(edge_groups, target_edge_groups)
    all_metrics.append(metrics)
    print(mesh, metrics)

Hannover_AM_left_initial_proc (0.8559911476446411, 0.4896358543417367, 0.8734722222222222)
Hannover_AZ_initial_proc (0.7673822940436827, 0.5946274955364389, 0.8265625)
Hannover_DK_initialSegmentation_proc (0.7134807108799306, 0.46434359805510533, 0.7704861111111111)
Hannover_GDPT_left_initial_2_proc (0.7877041306436119, 0.4886317907444668, 0.8235069444444445)
Hannover_MJ_links_initial_proc (0.8791849847818736, 0.6428392901774557, 0.9007638888888889)
Hannover_SE_Segm1_initial_proc (0.8364904297937583, 0.7218810465214099, 0.8852083333333334)
Hannover_WLAD_initial_proc (0.9248930317848411, 0.5716775599128541, 0.9317361111111111)
HRA_20180325_2_initial_proc (0.8592492674077864, 0.7069816984603466, 0.8949305555555556)
Kiel_BB_Pat20_seg_proc (0.890459215768505, 0.6548169684154883, 0.9093055555555556)
Kiel_BB_Pat2_initial.cut_proc (0.9500805801772764, 0.762598198888676, 0.9569791666666667)


In [144]:
mean_sum = 0
iou_0_sum, iou_1_sum = 0, 0
acc_sum = 0
for mesh, metric in zip(test_meshes, all_metrics):
    iou_0 = metric[0]
    iou_1 = metric[1]
    acc = metric[2]
    
    mean_iou = (iou_0 + iou_1) / 2
    
    iou_0_sum += iou_0
    iou_1_sum += iou_1
    
    acc_sum += acc
    mean_sum += mean_iou
    
    print(mesh, mean_iou)

Hannover_AM_left_initial_proc 0.6728135009931889
Hannover_AZ_initial_proc 0.6810048947900609
Hannover_DK_initialSegmentation_proc 0.588912154467518
Hannover_GDPT_left_initial_2_proc 0.6381679606940394
Hannover_MJ_links_initial_proc 0.7610121374796646
Hannover_SE_Segm1_initial_proc 0.7791857381575842
Hannover_WLAD_initial_proc 0.7482852958488475
HRA_20180325_2_initial_proc 0.7831154829340665
Kiel_BB_Pat20_seg_proc 0.7726380920919966
Kiel_BB_Pat2_initial.cut_proc 0.8563393895329762


In [141]:
print('Mean IOU', mean_sum/(len(all_metrics)))
print('IOU Class 0', iou_0_sum/(len(all_metrics)))
print('IOU Class 1', iou_1_sum/(len(all_metrics)))
print('Accuracy', acc_sum/(len(all_metrics)))

Mean IOU 0.7281474646989943
IOU Class 0 0.8464915792925908
IOU Class 1 0.6098033501053979
Accuracy 0.877295138888889


In [143]:
test_meshes_info = \
"""
Hannover_AM_left_initial.obj	32375	64822
Hannover_AZ_initial.obj	73982	148012
Hannover_DK_initialSegmentation.obj	22089	44234
Hannover_GDPT_left_initial_2.obj	91753	183562
Hannover_MJ_links_initial.obj	108692	217464
Hannover_SE_Segm1_initial.obj	19904	39848
Hannover_WLAD_initial.obj	63931	127998
HRA_20180325_2_initial.obj	56692	113388
Kiel_BB_Pat20_seg.obj	159325	318762
Kiel_BB_Pat2_initial.cut.obj	51989	104042
"""


In [ ]:
for sp in test_meshes_info.split('\n'):
    s = sp.split('\t')
    s[0]